In [1]:
import os, glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

from ultralytics import YOLO
import easyocr
import utils

In [2]:
if "temp" not in os.listdir():
    os.makedirs("./temp")

<h3 style="color:orange">Hyperparameters</h3>

In [3]:
DEVICE="cuda" if torch.cuda.is_available() else "cpu"

print(f"{DEVICE=}")

vid_dirs=["./dataset/video/demo.mp4","./dataset/video/IND_demo.mp4"]
vid_dir=vid_dirs[1]

vid=cv2.VideoCapture(vid_dir)

# video metadata
frame_count=int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
W=int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
H=int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
FPS=int(vid.get(cv2.CAP_PROP_FPS))

print(f"frame count: {frame_count}\nframe width: {W}\nframe hight: {H}\nFPS: {FPS}")


DEVICE='cuda'
frame count: 631
frame width: 1114
frame hight: 720
FPS: 30


<h3 style="color:orange">Load Model</h3>

In [4]:
model_weight=[path for path in glob.glob("./Model/weights/*.pt")]
print(model_weight)
model=YOLO(model_weight[1]).to(DEVICE)

matter_tracker=utils.sort.Sort()   #use for tracting object in continues frames

print(model)

['./Model/weights\\yolov8n.pt', './Model/weights\\YOLOv8_LPD_largeset.pt', './Model/weights\\YOLOv8_LPD_roboflow.pt']
YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1,

<h3 style="color:orange">model infernce</h3>

In [ ]:
results={}

if "video" not in os.listdir("./temp/"): os.makedirs("./temp/video")
# if "true_frame" not in os.listdir("./temp/"): os.makedirs("./temp/true_frame")
# video save
out_mp4=cv2.VideoWriter(f"./temp/video/temp_{os.path.splitext(os.path.basename(vid_dir))[0]}.mp4",cv2.VideoWriter_fourcc(*"mp4v"),FPS,(W,H))

LIMIT=50

ret,frame_num=True,-1
while(ret):
    frame_num+=1
    ret,frame=vid.read()
    # cv2.imwrite(f"./temp/true_frame/{frame_num}.png",frame)
    
    print(f"Frame no. {frame_num}")
    if(ret):

        # if(frame_num>LIMIT): break     #debug
        
        results[frame_num]={}

        # detect numberplate
        detections=model(frame)[0]
        detection_list=[]
        for detection in detections.boxes.data.tolist():
            x1,y1,x2,y2,score,class_id=detection
            if(score>=0.5):
                # print([int(x1),int(y1),int(x2),int(y2),score])
                detection_list.append([int(x1),int(y1),int(x2),int(y2),score])
        print("\n")
    

        ## track vehicle 
        # track_id=matter_tracker.update(np.asarray(detection_list))
        # print(track_id)

        # crop Lincence plate
        crop_license_plate=frame[int(y1):int(y2),int(x1):int(x2),:]
        # print(crop_license_plate.shape)
        if "crop" not in os.listdir("./temp/"): os.makedirs("./temp/crop")
        cv2.imwrite(f"./temp/crop/{frame_num}.png",crop_license_plate)    ## DEBUG

        ## correct one
        ans=utils.utils.read_license_plate_video(crop_license_plate)
        ans=np.array(ans)
        # print(ans,ans.shape)

        if(len(ans)==0):
            out_mp4.write(frame)        # to add blank predition files
            continue
        elif(ans.shape[0]!=1):
            ans=ans[np.newaxis,:]
        # wrting test sample in ans_img
        if "ans_img" not in os.listdir("./temp/"): os.makedirs("./temp/ans_img")

        # add color base on correct OCR identification
        # print(len(ans),len(ans[0]))

        for text,score,Flag in ans:
            print(Flag)
            color=(0,255,0) if Flag else (0,0,255)
            # bbox
            cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),(int(y2))),color,3)
            # text
            cv2.putText(frame,text,(int(x1),int(y1)-3),cv2.FONT_HERSHEY_PLAIN,2,color,2,cv2.LINE_AA)
            cv2.imwrite(f"./temp/ans_img/{os.path.splitext(os.path.basename(vid_dir))[0]}_{frame_num}.png",frame)    ## DEBUG

    ##  save video
    out_mp4.write(frame)

# clear all video caps
out_mp4.release()
cv2.destroyAllWindows()

Frame no. 0

0: 416x640 2 License Platess, 50.0ms
Speed: 3.1ms preprocess, 50.0ms inference, 97.6ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 1

0: 416x640 2 License Platess, 7.9ms
Speed: 1.7ms preprocess, 7.9ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 2

0: 416x640 2 License Platess, 7.4ms
Speed: 1.9ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 3

0: 416x640 3 License Platess, 7.1ms
Speed: 1.9ms preprocess, 7.1ms inference, 1.4ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 4

0: 416x640 3 License Platess, 8.6ms
Speed: 1.8ms preprocess, 8.6ms inference, 1.8ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 5

0: 416x640 3 License Platess, 7.6ms
Speed: 1.8ms preprocess, 7.6ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)


False
Frame no. 6

0: 416x640 3 License Platess, 8.4ms
Speed: 1.7ms preprocess, 8.4ms